## Core

> The building blocks to the UI

In [ ]:
#| default_exp core

## Imports

In [ ]:
#| hide
#| export
import fasthtml.common as fh
from fh_frankenui.foundations import *
from fasthtml.common import is_listy, Div, P, Span, Script, FastHTML, FT, to_xml, show,fast_app
from fasthtml.svg import Svg
from fasthtml.components import Uk_theme_switcher, Main

from enum import Enum, auto
from fasthtml.components import Uk_select,Uk_input_tag,Uk_icon
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional
from fastcore.all import *
import copy, re


In [ ]:
#| hide
from IPython.display import HTML, display, Markdown
from fasthtml.jupyter import *
from functools import partial


In [ ]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">


<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
</style>

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
#| export
def enum_to_markdown_table(enum_class):
    headers = ["Option", "Value"]
    rows = [[name, value.value] for name, value in enum_class.__members__.items()]
    
    max_name_len = max(len(row[0]) for row in rows)
    max_value_len = max(len(row[1]) for row in rows)
    
    header = f"| {'Option':<{max_name_len}} | {'Value':<{max_value_len}} |"
    separator = f"|{'-'*(max_name_len+2)}|{'-'*(max_value_len+2)}|"
    body = "\n".join(f"| {row[0]:<{max_name_len}} | {row[1]:<{max_value_len}} |" for row in rows)
    
    return f"{header}\n{separator}\n{body}"

# App

In [ ]:
#| export
@delegates(fh.fast_app)
def fast_app(*args, **kwargs):
    "Adds `bg-background text-foreground` to bodykw for frankenui themes"
    if 'bodykw' not in kwargs: kwargs['bodykw'] = {}
    if 'class' not in kwargs['bodykw']: kwargs['bodykw']['class'] = ''
    kwargs['bodykw']['class'] = stringify((kwargs['bodykw']['class'],'bg-background text-foreground'))
    return fh.fast_app(*args, **kwargs)

In [ ]:
assert fast_app(bodykw={'something':'test'})[0].bodykw == {'something': 'test', 'class': ' bg-background text-foreground'}

In [ ]:
#| export
@delegates(fh.FastHTML)
def FastHTML(*args, **kwargs):
    "Adds `bg-background text-foreground` to bodykw for frankenui themes"
    if 'bodykw' not in kwargs: kwargs['bodykw'] = {}
    if 'bodykw' not in kwargs['bodykw']: kwargs['bodykw']['class'] = ''
    kwargs['bodykw']['class'] = stringify((kwargs['bodykw']['class'],'bg-background text-foreground'))
    return fh.FastHTML(*args, **kwargs)

## Theme / Headers

You can select a theme color to and get all the headers.

In [ ]:
#| export
def _headers_theme(color):    
    return fh.Script(f'''const htmlElement = document.documentElement;
          if (
            localStorage.getItem("mode") === "dark" ||
            (!("mode" in localStorage) &&
              window.matchMedia("(prefers-color-scheme: dark)").matches)
          ) {{
            htmlElement.classList.add("dark");
          }} else {{
            htmlElement.classList.remove("dark");
          }}

          htmlElement.classList.add(localStorage.getItem("theme") || "uk-theme-{color}");
          ''')

In [ ]:
#| export
class Theme(Enum):
    "Selector to choose theme and get all headers needed for app.  Includes frankenui + tailwind"
    def _generate_next_value_(name, start, count, last_values): return name
    slate = auto()
    stone = auto()
    gray = auto()
    neutral = auto()
    red = auto()
    rose = auto()
    orange = auto()
    green = auto()
    blue = auto()
    yellow = auto()
    violet = auto()
    zinc = auto()
    
    def headers(self):
        return (fh.Link(rel="stylesheet", href="https://unpkg.com/franken-ui@1.1.0/dist/css/core.min.css"),
            fh.Script( type="module", src ="https://unpkg.com/franken-ui@1.1.0/dist/js/core.iife.js"),
            fh.Script( type="module", src ="https://unpkg.com/franken-ui@1.1.0/dist/js/icon.iife.js"),
            fh.Script(src="https://cdn.tailwindcss.com"),
            _headers_theme(self.value),)

In [ ]:
hdrs = Theme.blue.headers()
app = FastHTML(hdrs=hdrs)

In [ ]:
#| hide
def create_server(app,rt):
    if 'server' in globals(): globals()['server'].stop()
    if IN_NOTEBOOK:
        for port in range(8000,8030):   
            if is_port_free(port):
                server = JupyUvi(app, port=port)

                def HShow(comp, app):
                    @app.get('/')
                    def get(): return comp
                    display(HTML(f'<a href="http://localhost:{port}/" target="_blank">Open in new tab</a>'))
                    return HTMX("/",port=port)
                print(f"Starter server on port {port}")
                Show = partial(HShow, app=app)
                return app, rt, server, HShow, Show
            
if IN_NOTEBOOK: 
    app, rt = fast_app(pico=False, hdrs=Theme.blue.headers())
    app, rt, server, HShow, Show = create_server(app, rt)
else: 
    Show=noop

Starter server on port 8001


## Text Style

In [ ]:
#| export
class TextT(VEnum):
    'Text Styles from https://franken-ui.dev/docs/text'
    def _generate_next_value_(name, start, count, last_values):
        return str2ukcls('text', name)
    
    # Text Style
    lead,meta, italic = auto(), auto(), auto()
    # Text Size
    small, default, large = auto(), 'uk-text', auto()
    # Text Weight
    light, normal, bold, lighter, bolder = auto(),auto(),auto(),auto(),auto()
    # Text Transform
    capitalize,uppercase, lowercase = auto(),auto(),auto()
    # Text Decoration
    decoration_none = auto()
    # Text Color
    muted,primary,secondary, success,warning, danger = auto(),auto(),auto(),auto(),auto(),auto()
    # Text Alignment
    left, right,center,justify = auto(), auto(), auto(), auto()
    # Vertical Alignment
    top,middle,bottom, baseline = auto(),auto(),auto(),auto()
    
    # Text Wrapping
    truncate,break_,nowrap = auto(),auto(),auto()

In [ ]:
# print(enum_to_markdown_table(TextT))

In [ ]:
#| export
class TextFont(Enum):
    "Combinations of TextT that are particularly useful"
    def __add__(self, other):   return stringify((self, other))
    def __radd__(self, other):  return stringify((other, self)) 
    def __str__(self): return self.value
    muted_sm = stringify((TextT.muted, TextT.small))
    muted_lg = stringify((TextT.muted, TextT.large))
    bold_sm = stringify((TextT.bold, TextT.small))
    

## Button

In [ ]:
#| export
class ButtonT(VEnum):
    "Options for styling Buttons"
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('button', name)
    default = auto()
    primary = auto()
    secondary = auto()
    danger = auto()
    text = auto()
    link = auto()
    ghost = auto()

In [ ]:
#| export
def Button(*c: Union[str, FT],
           cls: Union[str, Enum]=ButtonT.default,
           submit=True
           **kwargs
          ) -> FT:
    "A Button with Styling (defaults to `type=submit` for form submission)"
    return fh.Button(*c, cls=('uk-button', stringify(cls)), type='submit' if submit else 'button', **kwargs)

In [ ]:
#| export
def ButtonSubmit(*c: Union[str, FT],
           cls: Union[str, Enum]=ButtonT.default,
           **kwargs
          ) -> FT:
    "A Button with Styling to be used for submitting forms"
    return fh.Button(*c, cls=('uk-button', stringify(cls)), type='submit', **kwargs)

In [ ]:
show(Div(Button("Default Class"),
        Button("Primary Class", cls=ButtonT.primary),
        Button("Secondary Class", cls=ButtonT.secondary),
        Button("Danger Class", cls=ButtonT.danger),
        Button("Ghost Class", cls=ButtonT.ghost),
        Button("Link Class", cls=ButtonT.link),
        Button("Text Class", cls=ButtonT.text),))

## Headings

In [ ]:
#| export
def H1(*c:FT|str, cls:Enum|str|tuple=(), **kwargs)->FT: 
    "A H1 with Styling"
    return fh.H1(*c, cls=('uk-h1',stringify(cls)), **kwargs)

In [ ]:
#| export
def H2(*c:FT|str, cls:Enum|str|tuple=(), **kwargs)->FT: 
    "A H2 with Styling"
    return fh.H2(*c, cls=('uk-h2',stringify(cls)), **kwargs)

In [ ]:
#| export
def H3(*c:FT|str, cls:Enum|str|tuple=(), **kwargs)->FT: 
    "A H3 with Styling"
    return fh.H3(*c, cls=('uk-h3',stringify(cls)), **kwargs)

In [ ]:
#| export
def H4(*c:FT|str, cls:Enum|str|tuple=(), **kwargs)->FT:
    "A H4 with Styling"
    return fh.H4(*c, cls=('uk-h4',stringify(cls)), **kwargs)

In [ ]:
#| export
def Main(*args, **kwargs): return Main(*args, **kwargs)

In [ ]:
#| export
def Titled(title:str="FastHTML app", *args, cls="container", **kwargs)->FT:
    return fh.Title(title), fh.Main(H1(title), *args, cls=cls, **kwargs)

In [ ]:
show(Div(
    Titled("Titled"),
    H1("Heading 1 (H1)"),
    H2("Heading 2 (H2)"),
    H3("Heading 3 (H3)"),
    H4("Heading 4 (H4)")))

## Dividers

In [ ]:
#| export
class DividerT(VEnum):
    " Divider Styles from https://franken-ui.dev/docs/divider"
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('divider', name)
    icon=auto()
    small=auto()
    vertical=auto()

In [ ]:
#| export
def Divider(*args, cls=('my-4', DividerT.icon), **kwargs):
    "Divider with default styling and margin"
    cls = stringify(cls)
    container = Div if 'uk-divider-vertical' in cls else Hr
    return container(*args, cls=cls, **kwargs)

In [ ]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    # Divider FrankenUI stuff
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

In [ ]:
show(UkHSplit())

In [ ]:
#| export
def UkHLine(lwidth=2, y_space=4): return Div(cls=f"my-{y_space} h-[{lwidth}px] w-full bg-secondary")

In [ ]:
show(UkHLine())

## Articles & Containers & Sections

In [ ]:
#| export
def Article(*c, cls=(), **kwargs):
    return fh.Article(*c, cls=('uk-article',stringify(cls)), **kwargs)

def ArticleTitle(*c, cls=(), **kwargs):
    return H1(*c, cls=('uk-article-title',stringify(cls)), **kwargs)

def ArticleMeta(*c, cls=(), **kwargs):
    return P(*c, cls=('uk-article-meta',stringify(cls)), **kwargs)

In [ ]:
show(Article(ArticleTitle("Article Title"), ArticleMeta("By: John Doe")))

In [ ]:
#| export
class ContainerT(VEnum):
    'Max width container sizes from https://franken-ui.dev/docs/container'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('container', name)
    xsmall = auto()
    small = auto()
    large = auto()
    xlarge = auto()
    expand = auto()

In [ ]:
#| export
def DivContainer(*c, cls=(), **kwargs): 
    "A Div to be used as a container that often wraps large sections or a page of content"
    return Div(*c, cls=('uk-container',stringify(cls)), **kwargs)

In [ ]:
show(DivContainer(
    "This is a container with a custom background color.",
    cls=ContainerT.xsmall,
    style="background-color: #e6f2ff; padding: 20px; border-radius: 5px;"
))

In [ ]:
#| export
class SectionT(VEnum):
    'Section styles from https://franken-ui.dev/docs/section'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('section', name)
    default = auto()
    muted = auto()
    primary = auto()
    secondary = auto()
    xsmall = auto()
    small = auto()
    large = auto()
    xlarge = auto()
    remove_vertical = auto()

In [ ]:
# Markdown(enum_to_markdown_table(SectionT))

In [ ]:
#| export
def Section(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-section',stringify(cls)), **kwargs)

## Forms & Inputs

In [ ]:
#|export
def Fieldset(*c, cls=(), **kwargs): 
    return fh.Fieldset(*c, cls=('uk-fieldset',stringify(cls)), **kwargs)

def Legend(*c, cls=(), **kwargs): 
    return fh.Legend(*c, cls=('uk-legend',stringify(cls)), **kwargs)

In [ ]:
#| export
def Input(*c, cls=(), **kwargs):      
    return fh.Input(*c, cls=('uk-input',stringify(cls)), **kwargs)
def Select(*option, cls=(), **kwargs):
    return fh.Select(*option, cls=('uk-select',stringify(cls)), **kwargs)
def Radio(*c, cls=(), **kwargs):      
    return fh.Input(*c, cls=('uk-radio',stringify(cls)), type='radio', **kwargs)
def CheckboxX(*c, cls=(), **kwargs):  
    return fh.Input(*c, cls=('uk-checkbox',stringify(cls)), type='checkbox', **kwargs)
def Range(*c, cls=(), **kwargs):      
    return fh.Input(*c, cls=('uk-range',stringify(cls)), type='range', **kwargs)
def Toggle_switch(*c, cls=(), **kwargs):
    return fh.Input(*c, cls=('uk-toggle-switch',stringify(cls)), type='checkbox', **kwargs)
def TextArea(*c, cls=(), **kwargs):            
    return fh.Textarea(*c, cls=('uk-textarea',stringify(cls)), **kwargs)
def Switch(*c, cls='min-w-9', **kwargs):              
    return fh.Input(*c, cls=('uk-toggle-switch',stringify(cls)), type='checkbox', **kwargs)

In [ ]:
#|export
def FormLabel(*c, cls=(), **kwargs): return fh.Label(*c, cls=('uk-form-label',stringify(cls)), **kwargs)

In [ ]:
show(Form(
    Fieldset(
        Legend("Fieldset Legend 1"),
        FormLabel("Input: ", Input()),    
        FormLabel("Select: ", Select(fh.Option("Option 1"), fh.Option("Option 2"))),
        FormLabel("Radio: ", Radio()),
        FormLabel("Checkbox: ", CheckboxX()),
        FormLabel("Button: ", Button("Click me")),
        cls='space-x-4'),   
    Fieldset(
        Legend("Fieldset Legend 2"),        
        FormLabel("Range: ", Range()),
        FormLabel("Toggle switch: ", Toggle_switch()),
        FormLabel("TextArea: ", TextArea()),
        FormLabel("Switch: ", Switch()),
        cls='space-x-4'),   
    cls="space-y-6"
))

In [ ]:
#| export
class LabelT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('label', name)
    primary = auto()
    secondary = auto()
    danger = auto()

In [ ]:
#| export
def Label(*c, cls=(), **kwargs):
    "FrankenUI labels, which look like pills"
    return fh.Label(*c, cls=('uk-label',stringify(cls)), **kwargs)

In [ ]:
show(Div(
    Label("Default"),
    Label("Primary",cls=LabelT.primary),
    Label("Secondary",cls=LabelT.secondary),
    Label("Default",cls=LabelT.danger),
))

In [ ]:
#| export
def UkFormSection(title, description, *c, button_txt='Update', outer_margin=6, inner_margin=6):
    return Div(cls=f'space-y-{inner_margin} py-{outer_margin}')(
        Div(H3(title), P(description, cls=TextFont.muted_sm)),
        UkHSplit(), *c,
        Div(Button(button_txt, cls=ButtonT.primary)) if button_txt else None)

## Labeled Inputs

Inputs of various types often go with a label.  Because of this we created functions to do this for you along with properly linking the `for` attribute from the lable to the input.  We also have some nice defaults, such as putting a little spacing between the label and the input

In [ ]:
#| export
def GenericLabelInput(
               label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               cls='',
               id='',
               input_fn=noop, 
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(cls=stringify(lbl_cls), fr=id)(label)
    inp = input_fn(id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(label, inp, cls=stringify(cls))
    return label, inp

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn','cls'])
def LabelInput(*args, cls='space-y-2', **kwargs): 
    "A Label and Input pair that provides default spacing and links/names them based on id"
    return GenericLabelInput(*args, cls=stringify(cls),input_fn=Input, **kwargs)

In [ ]:
#| export
def LabelRadio(label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               cls='flex items-center space-x-2',
               id='',
                **kwargs
                ):
    "A Label and Radio pair that provides default spacing and links/names them based on id"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(cls=stringify(lbl_cls), fr=id)(label)
    inp = Radio(id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(inp, label, cls=stringify(cls))
    return inp, label

In [ ]:
show(LabelRadio('My Label',id='myid'))

In [ ]:
#| export
def LabelCheckboxX(label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               cls='flex items-center space-x-2',
               id='',
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(cls=stringify(lbl_cls), fr=id)(label)
    inp = CheckboxX(id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(inp, label, cls=stringify(cls))
    return inp, label

In [ ]:
#| export
# @delegates(GenericLabelInput, but=['input_fn','cls'])
# def LabelCheckboxX(*args, cls='space-x-2', **kwargs): return GenericLabelInput(*args, cls=stringify(cls), input_fn=CheckboxX, **kwargs)

In [ ]:
show(LabelCheckboxX('My Label',id='myid'))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn','cls'])
def LabelRange(*args, cls='space-y-2', **kwargs): return GenericLabelInput(*args, cls=stringify(cls), input_fn=Range, **kwargs)

In [ ]:
show(LabelRange('My Label',id='myid'))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn','cls'])
def LabelTextArea(*args, cls='space-y-2', **kwargs): return GenericLabelInput(*args, cls=stringify(cls), input_fn=TextArea, **kwargs)

In [ ]:
show(LabelTextArea('mylabel',container=Form))

In [ ]:
#| export
@delegates(GenericLabelInput, but=['input_fn','cls'])
def LabelSwitch(*args, cls='space-x-2', **kwargs): return GenericLabelInput(*args, cls=stringify(cls), input_fn=Toggle_switch, **kwargs)

In [ ]:
show(LabelSwitch('mylabel',id='myid'))

In [ ]:
#| export
def LabelSelect(*option,
               label:str|FT,
               lbl_cls='',
               input_cls='',
               container=Div, 
               cls='space-y-2',
               id='',
                **kwargs
                ):
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UkTextArea` which is created for you in this library"
    if isinstance(label, str) or label.tag != 'label': 
        label = FormLabel(lbl_cls=stringify(lbl_cls), fr=id)(label)
    inp = Select(*option, id=id, cls=stringify(input_cls), **kwargs)        
    if container: return container(label, inp, cls=stringify(cls))
    return label, inp

In [ ]:
show(LabelSelect(Option(1),Option(2),label='My Label', id='myid'))

In [ ]:
#| export
def Options(*c,                    # Content for an `Option`
            selected_idx:int=None, # Index location of selected `Option`
            disabled_idxs:set=None # Idex locations of disabled `Options`
           ):
    "Helper function to wrap things into `Option`s for use in `UkSelect`"
    return [fh.Option(o,selected=i==selected_idx, disabled=disabled_idxs and i in disabled_idxs) for i,o in enumerate(c)]

In [ ]:
# list(map(to_xml,Options('option1','option2','option3', selected_idx=1, disabled_idxs={0,2})))

UK Select is a custom select that has more functionality than the basic select.  One particularly useful feature is the `searchable`, which allows the user to type to search for options in the Select.

In [ ]:
#| export
def UkSelect(*option,            # Options for the select dropdown (can use `Options` helper function to create)
             inp_cls=(),         # Additional classes for the select input
             cls=('space-y-2',), # Classes for the outer div
             id="",              # ID for the select input
             name="",            # Name attribute for the select input
             placeholder="",     # Placeholder text for the select input
             searchable=False,   # Whether the select should be searchable
             **kwargs):          # Additional arguments passed to Uk_select
    "Creates a select dropdown with uk styling"
    inp_cls, cls = map(stringify, (inp_cls, cls))
    select = Uk_select(*option, cls=inp_cls, uk_cloak=True, id=id, 
                       name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    return Div(cls=cls)(select)

In [ ]:
#| export
def LabelUkSelect(*option,            # Options for the select dropdown (can use `Options` helper function to create)
             label=(),           # String or FT component for the label
             lbl_cls=(),         # Additional classes for the label
             inp_cls=(),         # Additional classes for the select input
             cls=('space-y-2',), # Classes for the outer div
             id="",              # ID for the select input
             name="",            # Name attribute for the select input
             placeholder="",     # Placeholder text for the select input
             searchable=False,   # Whether the select should be searchable
             **kwargs):          # Additional arguments passed to Uk_select
    "Creates a select dropdown with uk styling"
    lbl_cls, inp_cls, cls = map(stringify, (lbl_cls, inp_cls, cls))
    if label: 
        lbl = FormLabel(cls=f'{lbl_cls}', fr=id)(label) 
    select = Uk_select(*option, cls=inp_cls, uk_cloak=True, id=id, 
                       name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    return Div(cls=cls)(lbl, select) if label else Div(cls=cls)(select)

In [ ]:
show(UkSelect(Option(1),Option(2),label='My Label', id='myid', searchable=True))

## Links

In [ ]:
#| export
class AT(VEnum):
    'Link styles from https://franken-ui.dev/docs/link'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('link', name)
    muted = auto()
    text = auto()
    reset = auto()

In [ ]:
#| export
# def Link(*c, cls=(), **kwargs):  
#     return fh.A(*c, cls=('uk-link',stringify(cls)), **kwargs)

In [ ]:
show(Div(cls='space-x-4')(('Default Link'),
                     A('Muted Link', cls=AT.muted),
                     A('Text Link', cls=AT.text),
                     A('Reset Link', cls=AT.reset)))

## Lists

In [ ]:
#| export
class ListT(VEnum):
    'List styles from https://franken-ui.dev/docs/list'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('list', name)
    disc = auto()
    circle = auto()
    square = auto()
    decimal = auto()
    hyphen = auto()
    muted = auto()
    primary = auto()
    secondary = auto()
    bullet = auto()
    divider = auto()
    striped = auto()

In [ ]:
#| export
def List(*c, cls=(), **kwargs): return fh.Ul(*c, cls=('uk-list',stringify(cls)), **kwargs)

In [ ]:
list_options = [(style,str(cls)) for style,cls in ListT.__members__.items()]
def create_list(style, cls): return List(fh.Li("Item 1"), fh.Li("Item 2"), cls=cls)

lists = [Div(H4(f"{style} List:"), create_list(style, cls)) for style, cls in list_options]
show(Div(*lists, cls="grid grid-cols-4"))

## Modal

In [ ]:
#| export
def ModalContainer(*c, cls=(), **kwargs):   return fh.Div(*c, cls=('uk-modal-container',stringify(cls)), uk_modal=True, **kwargs)
def ModalDialog(*c, cls=(), **kwargs):      return fh.Div(*c, cls=('uk-modal-dialog',   stringify(cls)),                **kwargs)
def ModalHeader(*c, cls=(), **kwargs):      return fh.Div(*c, cls=('uk-modal-header',   stringify(cls)),                **kwargs)
def ModalBody(*c, cls=(), **kwargs):        return fh.Div(*c, cls=('uk-modal-body',     stringify(cls)),                **kwargs)
def ModalFooter(*c, cls=(), **kwargs):      return fh.Div(*c, cls=('uk-modal-footer',   stringify(cls)),                **kwargs)
def ModalTitle(*c, cls=(), **kwargs):       return fh.H2(*c,  cls=('uk-modal-title',    stringify(cls)),                **kwargs)
def ModalCloseButton(*c, cls=(), **kwargs): return Button(*c, cls=('uk-modal-close',    stringify(cls)),                **kwargs)

In [ ]:
show(ModalCloseButton('Modal Close Button', cls=ButtonT.primary))

In [ ]:
#| export
def Modal(*c,
        header=None,          # Components that go in the header
        footer=None,          # Components that go in the footer
        cls=(),               # class for outermost container
        dialog_cls=(),        # classes for the dialog
        header_cls='p-6',     # classes for the header
        body_cls='space-y-6', # classes for the body
        footer_cls=(),        # classes for the footer
        id='',                # id for the outermost container
        **kwargs              # classes for the outermost container
        ): # Modal
    "Create a Modal using the appropriate Modal* classes to put the boilerplate in the appropriate places for you"
    cls, dialog_cls, header_cls, body_cls, footer_cls = map(stringify, (cls, dialog_cls, header_cls, body_cls, footer_cls))
    res = []
    if header: res.append(ModalHeader(cls=header_cls)(header))
    res.append(ModalBody(cls=body_cls)(*c))
    if footer: res.append(ModalFooter(cls=footer_cls)(footer))
    return ModalContainer(ModalDialog(*res, cls=dialog_cls), cls=cls, id=id, **kwargs)

In [ ]:
Show(Div(
    Button("Open Modal",uk_toggle="target: #my-modal" ),
    Modal(ModalTitle("Simple Test Modal"), 
          P("With some somewhat brief content to show that it works!", cls=TextFont.muted_sm),
          footer=ModalCloseButton("Close", cls=ButtonT.primary),id='my-modal')))

## Padding | Positioning

In [ ]:
#| export
class PaddingT(VEnum):
    'Padding Modifiers from https://franken-ui.dev/docs/padding'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('padding', name)
    xsmall = auto()
    small = auto()
    default = ''
    medium = auto()
    large = auto()
    xlarge = auto()
    remove = auto()
    remove_top = auto()
    remove_bottom = auto()
    remove_left = auto()
    remove_right = auto()
    remove_vertical = auto()
    remove_horizontal = auto()

In [ ]:
#| export
class PositionT(VEnum):
    'Position modifiers from https://franken-ui.dev/docs/position'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('position', name)
    top = auto()
    bottom = auto()
    left = auto()
    right = auto()
    top_left = auto()
    top_center = auto()
    top_right = auto()
    center = auto()
    center_left = auto()
    center_right = auto()
    bottom_left = auto()
    bottom_center = auto()
    bottom_right = auto()
    center_horizontal = auto()
    center_vertical = auto()

## Other

In [ ]:
#| export
def Placeholder(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-placeholder',stringify(cls)), **kwargs)

In [ ]:
show(Placeholder("Placeholder"))

In [ ]:
#| export
def Progress(*c, cls=(), value="", max="", **kwargs):
    return fh.Progress(*c, value=value, max=max, cls=('uk-progress',stringify(cls)), **kwargs)

In [ ]:
show(Progress(value=20, max=100))

## Icons and Avatars

In [ ]:
#| export
def UkIcon(icon,height=None,width=None,stroke_width=None,cls=(), **kwargs):
    return Uk_icon(icon=icon, height=height, width=width, stroke_width=stroke_width, cls=cls, **kwargs)

In [ ]:
#| export
def UkIconLink(icon, 
           height=None,
           width=None,
           stroke_width=None,
           cls=(), 
           button=False, 
           **kwargs):
    
    fn = fh.Button if button else fh.A
    return fh.A(cls=(f"uk-icon-{'button' if button else 'link'}", stringify(cls)), **kwargs)(
        UkIcon(icon=icon, height=height, width=width, stroke_width=stroke_width))

In [ ]:
class IconButtonT(VEnum):
    xsmall = 'uk-icon-button-xsmall'
    small  = 'uk-icon-button-small'
    outline= 'uk-icon-button-outline'

In [ ]:
#| export
def DiceBearAvatar(seed_name, # Seed name (ie 'Isaac Flath')
                   h=20,         # Height 
                   w=20,          # Width
                  ):          # Span with Avatar
    url = 'https://api.dicebear.com/8.x/lorelei/svg?seed='
    return Span(cls=f"relative flex h-{h} w-{w} shrink-0 overflow-hidden rounded-full bg-accent")(
            fh.Img(cls=f"aspect-square h-{h} w-{w}", alt="Avatar", src=f"{url}{seed_name}"))

In [ ]:
show(DiceBearAvatar('Isaac Flath', 20, 20))

## Flexbox | Grid

The most common pattern for spacing it to organize the general high level page layout with a `Grid`, and smaller components with `Flex`.

:::{.callout-tip}
Play [Flex Box Froggy](https://flexboxfroggy.com/) to get an understanding of flex box.
:::

In [ ]:
#| export
class FlexT(VEnum):
    'Flexbox modifiers from UIkit'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('flex', name)
    
    # Display
    block, inline = 'uk-flex', auto()
    # Horizontal Alignment
    left, center, right, between, around = auto(), auto(), auto(), auto(), auto()
    # Vertical Alignment
    stretch, top, middle, bottom = auto(), auto(), auto(), auto()
    # Direction
    row, row_reverse, column, column_reverse = auto(), auto(), auto(), auto()
    # Wrap
    nowrap, wrap, wrap_reverse = auto(), auto(), auto()

In [ ]:
#|export
def Grid(*div,         # Divs/Containers that should be divided into a grid
         cols_min=None, # defaults to min(len(div),1)
         cols_sm=None, # defaults to min(len(div),2)
         cols_md=None, # defaults to min(len(div),3)
         cols_lg=None, # defaults to min(len(div),4)
         cols=None,    # Size at every breakpoint
         cls='gap-4',  # Additional classes for Grid Div
         **kwargs      # Additional args for Grid Div
        ):
    """Creates a grid with the given number of columns, often used for a grid of cards"""
    cols_min = cols_min if cols_min else cols if cols else 1
    cols_sm = cols_sm if cols_sm else cols if cols else max(min(len(div), 2), cols_min+1)
    cols_md = cols_md if cols_md else cols if cols else max(min(len(div), 3), cols_min+2)
    cols_lg = cols_lg if cols_lg else cols if cols else max(min(len(div), 3), cols_min+2)
    cls = stringify(cls)
    return Div(cls=(f'grid grid-cols-{cols_min} sm:grid-cols-{cols_sm} md:grid-cols-{cols_md} lg:grid-cols-{cols_lg}',cls), **kwargs)(*div)

We can use the `FlexT` enum above to create `Div`s that arrange components in a flex box in different ways.  

For example, `DivFullySpaced` applies a very common pattern so we've made a function that helps to this for you.  But if you look at the code, it's really just using the enum to apply the right combonation of flex classes.

In [ ]:
#| exports
def DivFullySpaced(*c,                # Components
                   cls='uk-width-1-1',# Classes for outer div
                   **kwargs           # Additional args for outer div
                  ):                  # Div with spaced components via flex classes
    "Creates a flex div with it's components having as much space between them as possible"
    cls = stringify(cls)
    return Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
from functools import partial
_button = partial(Button,cls=ButtonT.primary)

In [ ]:
show(DivFullySpaced(*map(_button, range(3))))

In [ ]:
#| export
def DivCentered(*c,      # Components
                cls='space-y-4',  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components centered in it
    "Creates a flex div with it's components centered in it"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.column,FlexT.middle,FlexT.center,cls),**kwargs)(*c)

In [ ]:
show(DivCentered(*map(_button, range(3))))

In [ ]:
#| export
def DivLAligned(*c,      # Components
                cls='space-x-4',  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the left
    "Creates a flex div with it's components aligned to the left"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.left,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
show(DivLAligned(*map(_button, range(3))))

In [ ]:
#| export
def DivRAligned(*c,      # Components
                cls='space-x-4',  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the right
    "Creates a flex div with it's components aligned to the right"
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.right,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
show(DivRAligned(*map(_button, range(3))))

In [ ]:
#| export
def DivVStacked(*c, cls='space-y-4', **kwargs):
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.column,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
show(DivVStacked(*map(_button, range(3))))

In [ ]:
#| export
def DivHStacked(*c, cls='space-x-4', **kwargs):
    cls=stringify(cls)
    return Div(cls=(FlexT.block,FlexT.row,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
show(DivHStacked(*map(_button, range(3))))

## Nav

A `Nav` is something highly versatile that is the foundation for many things.  Three common places to use a Nav:

    + On its own as a sidebar navigation
    + In a button to give it a drop-down like behavior
    + In a Navbar to give drop down options in the navbar

In [ ]:
#| export
class NavT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('nav', name)
    default = auto()
    primary = auto()
    secondary = auto()

In [ ]:
#| export
def NavContainer(*li, 
                 cls=NavT.primary,
                 parent=True, 
                 uk_nav=False, #True for default collapsible behavior, see https://franken-ui.dev/docs/nav#component-options for more advanced options
                 **kwargs):
    return fh.Ul(*li, uk_nav=uk_nav, cls=(f"uk-nav{'' if parent else '-sub'}", stringify(cls)),**kwargs)

In [ ]:
mbrs1 = [Li(A('Option 1'), cls='uk-active'), 
         Li(A('Option 2')),
         Li(A('Option 3'))]

In [ ]:
show(NavContainer(*mbrs1))

In [ ]:
#| export
def NavParentLi(*nav_container, cls=(), **kwargs): return fh.Li(*nav_container,  cls=('uk-parent',  stringify(cls)),**kwargs)
def NavDividerLi(*c,cls=(), **kwargs): return fh.Li(*c, cls=('uk-nav-divider', stringify(cls)),**kwargs)
def NavHeaderLi(*c,cls=(), **kwargs): return fh.Li(*c, cls=('uk-nav-header', stringify(cls)),**kwargs)
def NavSubtitle(*c,cls=TextFont.muted_sm, **kwargs): return fh.Div(*c, cls=('uk-nav-subtitle', stringify(cls)),**kwargs)
def NavCloseLi(*c,cls=(), **kwargs): return fh.Li(*c, cls=('uk-drop-close', stringify(cls)),**kwargs)

In [ ]:
NavDividerLi()

```html
<li class="uk-nav-divider "></li>

```

In [ ]:
mbrs2 = [Li(A('Child 1')), Li(A('Child 2')),Li(A('Child 3'))]

Show(
    NavContainer(
        NavHeaderLi("NavHeaderLi"),
        *mbrs1,
        Li(A(href='')(Div("Subtitle Ex",NavSubtitle("NavSubtitle text to be shown")))),
        NavDividerLi(),
        NavParentLi(
            A('Parent Name'),
            NavContainer(*mbrs2,parent=False),
             ),
    )
)

In [ ]:
def NavParentIcon(): return Span(uk_nav_parent_icon=True)

## NavBar

In [ ]:
#| export
def NavBarContainer(*c, 
                    cls=(),
                    container_cls=ContainerT.expand,
                    uk_navbar=True,
                    **kwargs): 
    return fh.Div(DivContainer(Div(*c, uk_navbar=uk_navbar),cls=stringify(container_cls)), cls=('',stringify(cls)), **kwargs) #uk-navbar-container
def NavBarLSide(*c,  cls=(), **kwargs): return fh.Div(*c, cls=('uk-navbar-left',  stringify(cls)), **kwargs)
def NavBarRSide(*c,  cls=(), **kwargs): return fh.Div(*c, cls=('uk-navbar-right', stringify(cls)), **kwargs)
def NavBarCenter(*c, cls=(), **kwargs): return fh.Div(*c, cls=('uk-navbar-center',stringify(cls)), **kwargs)

In [ ]:
#| export
def NavBarNav(*li, cls=(), **kwargs): return fh.Nav(*li, cls=('uk-navbar-nav',      stringify(cls)),                 **kwargs)

In [ ]:
#| export
def NavBarSubtitle(title, subtitle, cls=(), subtitle_cls=TextFont.muted_sm, **kwargs): 
    return fh.Div(title,fh.Div(subtitle, cls=('uk-navbar-subtitle', stringify(subtitle_cls))), cls=stringify(cls), **kwargs)

In [ ]:
#| export
def NavBarNavContainer(*li, 
                       cls=NavT.primary,
                         parent=True, 
                         uk_nav=False, #True for default collapsible behavior, see https://franken-ui.dev/docs/nav#component-options for more advanced options
                         **kwargs):
    return Div(cls="uk-navbar-dropdown")(NavContainer(*li, cls=('uk-navbar-dropdown-nav',stringify(cls)), uk_nav=uk_nav, parent=parent, **kwargs))

In [ ]:
#|export
def NavBarParentIcon(): return Span(uk_navbar_parent_icon=True)

In [ ]:
l = NavBarNav(Li(cls='uk-active')(A("Active",href='')),
              Li(A("Parent",href=''),
                  NavBarNavContainer(
                      Li(cls='uk-active')(A("Active",href='')),
                      Li(A("Item",href='')),
                      Li(A("Item",href='')))),
              Li(A("Item",href='')))

r = NavBarNav(
            Li(cls='uk-active')(A(NavBarSubtitle("Title","Subtitle"),href='')),
            Li(A("DropDown",NavBarParentIcon(),href=''),
                NavBarNavContainer(
                    NavHeaderLi("NavHeaderLi"),
                    *mbrs1,
                    Li(A(href='')(Div("Subtitle Ex",NavSubtitle("NavSubtitle text to be shown")))),
                    NavDividerLi(),
                    NavParentLi(
                        A('Parent Name'),
                        NavContainer(*mbrs2,parent=False)))),
            
    Li(A(NavBarSubtitle("Title","Subtitle"),href='')))

In [ ]:
Show(
NavBarContainer(
    NavBarLSide(l),
    NavBarRSide(r)
)
)

## DropDown

In [ ]:
#| export
def DropDownNavContainer(*li, 
                         cls=NavT.primary,
                         parent=True, 
                         uk_nav=False, #True for default collapsible behavior, see https://franken-ui.dev/docs/nav#component-options for more advanced options
                         uk_dropdown=True,
                         **kwargs):
    return Div(cls = 'uk-drop uk-dropdown',uk_dropdown=uk_dropdown)(NavContainer(*li, cls=('uk-dropdown-nav',stringify(cls)), uk_nav=uk_nav, parent=parent, **kwargs))

In [ ]:
Show(Div(
        Button("Open DropDown"),
        DropDownNavContainer(Li(A("Item 1",href=''),Li(A("Item 2",href=''))))))

## Tabs

In [ ]:
#| export
def TabContainer(*li,cls='', alt=False, **kwargs):
    cls = stringify(cls)
    return Ul(cls=(f"uk-tab{'-alt' if alt else ''}",stringify(cls)),**kwargs)(*li)

In [ ]:
show(DivContainer(TabContainer(
    Li(A("Active",href='#', cls='uk-active')),
    Li(A("Item",href='#')),
    Li(A("Item",href='#')),
    Li(A("Disabled",href='#', cls='uk-disabled')),
    alt=True)))

In [ ]:
show(DivContainer(TabContainer(
    Li(A("Active",href='#', cls='uk-active')),
    Li(A("Item",href='#')),
    Li(A("Item",href='#')),
    Li(A("Disabled",href='#', cls='uk-disabled')),
    )))

## Cards

In [ ]:
#| export
class CardT(VEnum):
    'Card styles from UIkit'
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('card', name)
    default = auto()
    primary = auto()
    secondary = auto()
    danger = auto()

In [ ]:
#| export
def CardTitle(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-title',stringify(cls)), **kwargs)

def CardTitle(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-title',stringify(cls)), **kwargs)

def CardHeader(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-header',stringify(cls)), **kwargs)

def CardBody(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-body',stringify(cls)), **kwargs)

def CardFooter(*c, cls=(), **kwargs):
    return fh.Div(*c, cls=('uk-card-footer',stringify(cls)), **kwargs)

def CardContainer(*c, cls=CardT.default, **kwargs):
    return fh.Div(*c, cls=('uk-card',stringify(cls)), **kwargs)

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res.append(CardHeader(cls=header_cls)(header))
    res.append(CardBody(cls=body_cls)(*c))
    if footer: res.append(CardFooter(cls=footer_cls)(footer))
    return CardContainer(cls=cls, **kwargs)(*res)

In [ ]:
show(
    Card(
        Form(LabelInput("Input"),LabelRange("Range")),
        header=Div(CardTitle("Header"),P("A card with header and footer",cls=TextFont.muted_sm)),
        footer=DivLAligned(Button("Footer Submit button")),
)
)

## Tables

In [ ]:
#| export
class TableT(VEnum):
    def _generate_next_value_(name, start, count, last_values): return str2ukcls('table', name)
    divider = auto()
    striped = auto()
    hover = auto()
    small = auto()
    large = auto()
    justify = auto()
    middle = auto()
    responsive = auto()

In [ ]:
#| export
def Table(*args, cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), **kwargs): 
    return fh.Table(cls=('uk-table', stringify(cls)), *args, **kwargs)

In [ ]:
#| export
def _TableCell(Component, *args, cls=(), shrink=False, expand=False, small=False, **kwargs):
    cls = stringify(cls)
    if shrink: cls += ' uk-table-shrink'
    if expand: cls += ' uk-table-expand'
    if small: cls += ' uk-table-small'
    return Component(*args,cls=cls, **kwargs)

@delegates(_TableCell, but=['Component'])
def Td(*args,**kwargs):  return _TableCell(fh.Td, *args, **kwargs)
@delegates(_TableCell, but=['Component'])
def Th(*args,**kwargs): return _TableCell(fh.Th, *args, **kwargs)

def Tr(*cells, cls=(), **kwargs):  return fh.Tr(*cells, cls=stringify(cls), **kwargs)
def Thead(*rows, cls=(), **kwargs): return fh.Thead(*rows, cls=stringify(cls), **kwargs)
def Tbody(*rows, cls=(), sortable=False, **kwargs): return fh.Tbody(*rows, cls=stringify(cls), uk_sortable=sortable, **kwargs)

In [ ]:
show(
Table(
    Thead(Tr(Th('Name'), Th('Age'), Th('City'))),
    Tbody(Tr(Td('Alice'), Td('25'), Td('New York')),
          Tr(Td('Bob'), Td('30'), Td('San Francisco')),
          Tr(Td('Charlie'), Td('35'), Td('London'))),
    Tfoot(Tr(Td('Total'), Td('90'))))
)

In [ ]:
#|export
def TableFromLists(header_data, body_data, footer_data=None, 
                   header_cell_render=Th,body_cell_render=Td, footer_cell_render=Td,
                   cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small), 
                   sortable=False, **kwargs):
    
    return Table(
                Thead(Tr(*map(header_cell_render, header_data))),
                Tbody(*[Tr(*map(body_cell_render, r)) for r in body_data], sortable=sortable),
                Tfoot(Tr(*map(footer_cell_render, footer_data))) if footer_data else '',
                cls=stringify(cls),    
                **kwargs)

In [ ]:
# Test the function
header = ['Name', 'Age', 'City']
body = [['Alice', '25', 'New York'],
        ['Bob', '30', 'San Francisco'],
        ['Charlie', '35', 'London']]
footer = ['Total', '90']

show(TableFromLists(header, body, footer))

In [ ]:
#| export
def TableFromDicts(header_data:Sequence, body_data:Sequence[dict], footer_data=None, 
                   header_cell_render=Th, body_cell_render=lambda k,v : Td(v), footer_cell_render=lambda k,v : Td(v),
                   cls=(TableT.middle, TableT.divider, TableT.hover, TableT.small),
                   sortable=False,
                   **kwargs):
    
    return Table(
        Thead(Tr(*[header_cell_render(h) for h in header_data])),
        Tbody(*[Tr(*[body_cell_render(k, r.get(k, '')) for k in header_data]) for r in body_data], sortable=sortable),
        Tfoot(Tr(*[footer_cell_render(k, footer_data.get(k.lower(), '')) for k in header_data])) if footer_data else '',
        cls=stringify(cls),    
        **kwargs
    )

In [ ]:
header_data = ['Name', 'Age', 'City']
body_data = [{'Name': 'Alice', 'Age': 30, 'City': 'New York'},
             {'Name': 'Bob', 'Age': 25, 'City': 'London'}]

show(TableFromDicts(header_data, body_data))

Name,Age,City
Alice,30,New York
Bob,25,London


In [ ]:
# Example usage:
def header_render(v): return Th(v.upper())
def body_render(k, v):
    match k.lower():
        case 'name': return Td(v, cls='font-bold')
        case 'age':  return Td(f"{v} years")
        case _:      return Td(v)

header_data = ['Name', 'Age', 'City']
body_data = [{'Name': 'Alice', 'Age': 30, 'City': 'New York'},
             {'Name': 'Bob', 'Age': 25, 'City': 'London'}]

show(TableFromDicts(header_data, body_data, header_cell_render=header_render, body_cell_render=body_render))

NAME,AGE,CITY
Alice,30 years,New York
Bob,25 years,London


In [ ]:
# Example usage:
def header_render(v): return Th(v.upper())
def body_render(k, v):
    match k.lower():
        case 'name': return Td(v, cls='font-bold')
        case 'age': return Td(f"{v} years")
        case _: return Td(v)

header_data = ['Name', 'Age', 'City']
body_data = [{'Name': 'Alice', 'Age': 30, 'City': 'New York'},
             {'Name': 'Bob', 'Age': 25, 'City': 'London'}]

show(TableFromDicts(header_data, body_data, header_cell_render=header_render, body_cell_render=body_render))

NAME,AGE,CITY
Alice,30 years,New York
Bob,25 years,London


## Markdown

In [ ]:
#| export
def apply_classes(html_str:str, 
                  class_map=None, 
                  class_map_mods=None):
    from lxml import html, etree
    if not class_map:
        class_map = {
            'h1': 'uk-h1 my-4 mb-4',
            'h2': 'uk-h2 my-8',
            'h3': 'uk-h3 my-8',
            'h4': 'uk-h4 my-8',
            'ul': "uk-list uk-list-disc mb-4",
            'a': "uk-link",
            'p': 'my-1',
            'blockquote': "uk-blockquote mb-8",
            'hr':'uk-divider-icon my-4',
            'table':'uk-table-middle uk-table-divider uk-table-hover uk-table-small'
        }

    if class_map_mods: class_map = {**class_map, **class_map_mods}
    html_str = html.fromstring(html_str)
    for tag, classes in class_map.items():
        for element in html_str.xpath(f'//{tag}'):
            existing_class = element.get('class', '')
            new_class = f"{existing_class} {classes}".strip()
            element.set('class', new_class)
    return etree.tostring(html_str, encoding='unicode', method='html')

In [ ]:
_test = apply_classes("<div><h1>Hello, world!</h1></div>", class_map_mods={'h1': 'uk-h1 my-4 mb-4'})
assert _test == '<div><h1 class="uk-h1 my-4 mb-4">Hello, world!</h1></div>'

In [ ]:
#| export
def render_md(md_content):
    try: import markdown2
    except ImportError: raise ImportError("Install 'markdown2' to use the 'render_md' function")
    html_content = markdown2.markdown(md_content)
    styled_html = apply_classes(html_content)
    return NotStr(styled_html)

In [ ]:
def render_md(md_content):
    try:
        html_content = markdown2.markdown(md_content)
        styled_html = apply_classes(html_content)
        return NotStr(styled_html)
    except Exception as e:
        print(f"Error rendering markdown: {str(e)}", file=sys.stderr)
        return None

## _

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
%%html
<script>
window.addEventListener('load', function() {
    for (let i = 1; i <= 6; i++) {
        document.querySelectorAll(`h${i}`).forEach(el => {
            el.classList.add(`uk-h${i}`);
        });
    }
});

document.querySelectorAll('.sidebar-item-text').forEach(el => {
  el.style.visibility = 'visible';
})

</script>